09/05/2023
Notes:
- 3 inputs; Previous Q InRe Named TPE file, New Q InRe Named TPE file, New Q Treaty Register
- Current status: Named TPE totals incorrect

10/05/2023
Meeting Notes w/ A.Willaims:
- Add filter for UW, Cedant/BalloonID; important to keep senstitive information seperate secure. Filter will help this. 

# Libraries

In [23]:
# Import Libraries
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
import pandas as pd
import numpy as np
import xlrd

# Source Files

In [24]:
# InRe Named Exposure (Source Files)

                                ### Important the two file paths above are updated with new exposure paths ###       
Prev_Exposure = pd.read_excel(r'S:\Non Group Re\Actuarial\Gerard Coffey\Named Exposure\2023\INRE Named TPE April 2023 (April FX).xlsx')
New_Exposure  = pd.read_excel(r'S:\Non Group Re\Actuarial\Gerard Coffey\Named Exposure\2023\INRE Named TPE June 2023 (June FX).xlsx')
                                ### Important the two file paths above are updated with new exposure paths ### 

print(len(Prev_Exposure))
print(len(New_Exposure))

# Adjusted Treaty Register (Source Files)

# sheet_name selects the exact sheet from the excel file
# skiprows skips to the header rows in the file so we only take the necessary data
TreatyRegister = pd.read_excel(r'S:\Non Group Re\Actuarial\Gerard Coffey\Monthly Exposure Reporting\2023\MER\06\Inward Re MER Adjustment June 2023.xlsx', sheet_name = 'Adjustments', skiprows=6)
TreatyRegister.head()

20724
21294


,Reference,Balloon ID,TTY Status,Valid TTY,Bond/Credit,Latest Live UWY,Lapsed months,Run-off Factor,Point in Time Signed %,FX,...,Ultimate Costs,Incurred Losses,Ultimate Combined,Class of Business,Country,30/06/2023,Delta,Cedant.1,Short.1,Unnamed: 228
0,00779B1 0107,00779B1 01,New,1,BOND,Latest UWY,186,0.0,0.50,3.4528,...,33932.5409,462.47,6.603089e+04,Bond,Lithuania,0.000000e+00,0.0,BTA DRAUDIMAS,BTA Draudimas Q/S,NaN
1,00779B2 0308,00779B2 03,XL,1,BOND,Latest UWY,171,0.0,0.50,3.4528,...,5647.5900,0.00,1.115399e+04,Bond,Lithuania,0.000000e+00,0.0,BTA DRAUDIMAS,BTA Draudimas XL,NaN
2,00779B2 0307,00779B2 03,XL,1,BOND,NaN,183,0.0,0.50,3.4528,...,3973.0140,68497.31,8.740631e+04,Bond,Lithuania,0.000000e+00,0.0,BTA DRAUDIMAS,BTA Draudimas XL,NaN
3,00794B1 0123,00794B1 01,Renewed,1,BOND,Latest UWY,no lapse,1.0,0.24,1.0000,...,519009.1200,0.00,1.199409e+06,Bond,Latvia,1.197666e+08,0.0,BTA LATVIA,BTA Latvia Const QS,NaN
4,00794B1 0122,00794B1 01,Renewed,1,BOND,NaN,6,0.8,0.24,1.0000,...,494294.4000,61411.20,1.142294e+06,Bond,Latvia,0.000000e+00,0.0,BTA LATVIA,BTA Latvia Const QS,NaN


In [33]:
df = pd.concat([Prev_Exposure, New_Exposure], axis = 0, ignore_index=True)
print(len(df))
df.head()

42018


,KEY,LEGACY_POLICY_ID,SYMPHONY_POLICY_ID,CREDIT_LIMIT_ID,CLD_AMT_ORIGINAL_CURR,POLICY_CURRENCY,FIXED_EUR_EXC_RATE,VAR_EUR_EXC_RATE,CLD_TOTAL_AMT_EUR_FIXED,CLD_TOTAL_AMT_EUR_VAR,...,PARENT_REG_NUMBER,PARENT_VAT_NUMBER,PARENT_SRM_FLAG,PARENT_IBR_RATING,PARENT_RATING_BAND,CUSTOMER_ID,CEDANT_NAME,SHORT_NAME,COMMITMENT_CATEGORY,BuyerBalloon
0,00794B1 01,00794B1 01,1046306,119386691,11764000.0,EUR,1.0,1.0,11764000.0,11764000.0,...,44103125226,LV44103125226,N,74,71-80,23190842,BTA LATVIA,BTA Latvia Const QS,14,2341678600794B1 01
1,00794B1 01,00794B1 01,1046306,119386692,4944000.0,EUR,1.0,1.0,4944000.0,4944000.0,...,40103819319,LV40103819319,N,74,71-80,23190842,BTA LATVIA,BTA Latvia Const QS,14,4852247900794B1 01
2,00794B1 01,00794B1 01,1046306,119386693,3108000.0,EUR,1.0,1.0,3108000.0,3108000.0,...,40003146013,LV40003146013,N,56,51-60,23190842,BTA LATVIA,BTA Latvia Const QS,14,1489464800794B1 01
3,00794B1 01,00794B1 01,1046306,119386694,2278000.0,EUR,1.0,1.0,2278000.0,2278000.0,...,40003000252,LV40003000252,N,49,41-50,23190842,BTA LATVIA,BTA Latvia Const QS,14,2059524100794B1 01
4,00794B1 01,00794B1 01,1046306,119386695,1739000.0,EUR,1.0,1.0,1739000.0,1739000.0,...,40203228238,LV40203228238,N,48,41-50,23190842,BTA LATVIA,BTA Latvia Const QS,14,1558591700794B1 01


# Data Transformation

In [35]:
# Merger File
# Merge two files into one for BuyerReport Preparation
#df = pd.concat([Prev_Exposure, New_Exposure], ignore_index=True)
df["BUYER_ID"] = df["BUYER_ID"].apply(str)
df["BuyerBalloon"] = df["BUYER_ID"] + df["KEY"]

count_1 = df.BUYER_ID.size
print("Datapoints: " + str(count_1))

# How many unique Buyer ID's are in the combined df
count = df.BUYER_ID.unique().size
print("Number of Buyers: " + str(count))

Prev_Exposure["BUYER_ID"] = Prev_Exposure["BUYER_ID"].apply(str)
Prev_Exposure["BuyerBalloon"] = Prev_Exposure["BUYER_ID"] + Prev_Exposure["KEY"]

New_Exposure["BUYER_ID"] = New_Exposure["BUYER_ID"].apply(str)
New_Exposure["BuyerBalloon"] = New_Exposure["BUYER_ID"] + New_Exposure["KEY"]

#DecIBR["BUYER_ID"] = DecIBR["BUYER_ID"].apply(str)


Datapoints: 42018
Number of Buyers: 17400


# File Production

In [95]:
# Dataframe Creation

COLUMN_NAMES = ['BuyerBalloon',
                'Buyer_ID',
                'Buyer_Name',
                'Buyer_Country',
                'Buyer_Industry',
                'Buyer_TPE_Prev',
                'Buyer_TPE_New',
                'Buyer_TPE_Delta',
                'Buyer_IBR_Prev',
                'Buyer_IBR_New',
                'Buyer_IBR_Delta',
                'Parent_ID',
                'Parent_Name',
                'Parent_IBR_Prev',
                'Parent_IBR_New',
                'Parent_IBR_Delta', 
                'Balloon_ID', 
                'Cedant',
                'Underwriter'
]

BuyerReport = pd.DataFrame(columns=COLUMN_NAMES, index=None)


# Unique Balloon ID's being the reference point 
BuyerReport['BuyerBalloon'] = df.BuyerBalloon.unique()

#Mapping BuyerID's with corrosponding BalloonID in dataframe
BuyerID = dict(df[['BuyerBalloon', 'BUYER_ID']].values)
BuyerReport['Buyer_ID'] = BuyerReport['BuyerBalloon'].map(BuyerID)

# BuyerName 
BuyerName = dict(df[['BuyerBalloon', 'BUYER_NAME']].values)
BuyerReport['Buyer_Name'] = BuyerReport['BuyerBalloon'].map(BuyerName)

# Buyer Country
BuyerCountry = dict(df[['BuyerBalloon', 'BUYER_COUNTRY']].values)
BuyerReport['Buyer_Country'] = BuyerReport['BuyerBalloon'].map(BuyerCountry)

# Buyer Industry
BuyerIndustry = dict(df[['BuyerBalloon', 'BUYER_INDUSTRY']].values)
BuyerReport['Buyer_Industry'] = BuyerReport['BuyerBalloon'].map(BuyerIndustry)

# Previous Months TPE
TPE_Prev = Prev_Exposure.groupby('BuyerBalloon', as_index=False).agg({'CLD_TOTAL_AMT_EUR_VAR':'sum'})
Prev_Mapping = dict(TPE_Prev.values)
BuyerReport['Buyer_TPE_Prev'] = BuyerReport['BuyerBalloon'].map(Prev_Mapping)
BuyerReport["Buyer_TPE_Prev"].fillna(0, inplace=True)
BuyerReport = BuyerReport.astype({'Buyer_TPE_Prev':'int'})

# New Months TPE
TPE_New = New_Exposure.groupby('BuyerBalloon', as_index=False).agg({'CLD_TOTAL_AMT_EUR_VAR':'sum'})
New_Mapping = dict(TPE_New.values)
BuyerReport['Buyer_TPE_New'] = BuyerReport['BuyerBalloon'].map(New_Mapping)
BuyerReport['Buyer_TPE_New'].fillna(0, inplace=True)
BuyerReport = BuyerReport.astype({'Buyer_TPE_New':'int'})

# TPE Delta, subtracting movements. 
BuyerReport['Buyer_TPE_Delta'] = BuyerReport['Buyer_TPE_New'] - BuyerReport['Buyer_TPE_Prev']

## THE IBR RATINGS!!!! ##
##### Want to update all the IBR Ratings at once #####

Prev_Rating = dict(Prev_Exposure[['BuyerBalloon', 'BUYER_IBR_RATING']].values)
BuyerReport['Buyer_IBR_Prev'] = BuyerReport['BuyerBalloon'].map(Prev_Rating)

#Prev_Rating = dict(DecIBR[['BUYER_ID', 'BUYER_IBR_RATING']].values)
#BuyerReport['Buyer IBR Prev'] = BuyerReport['Buyer ID'].map(Prev_Rating)

New_Rating = dict(New_Exposure[['BuyerBalloon', 'BUYER_IBR_RATING']].values)
BuyerReport['Buyer_IBR_New'] = BuyerReport['BuyerBalloon'].map(New_Rating)


# Buyer IBR Delta
#BuyerReport['Buyer_IBR_New'].astype(float)
#BuyerReport['Buyer_IBR_Old'].astype(float)
#BuyerReport['Buyer_IBR_Delta'] = BuyerReport['Buyer_IBR_New'] - BuyerReport['Buyer_IBR_Prev']

# Parent ID 
ParentID = dict(df[['BuyerBalloon', 'PARENT_ID']].values)
BuyerReport['Parent_ID'] = BuyerReport['BuyerBalloon'].map(ParentID)

# Parent Name
ParentName = dict(df[['BuyerBalloon', 'PARENT_NAME']].values)
BuyerReport['Parent_Name'] = BuyerReport['BuyerBalloon'].map(ParentName)

# Parent IBR Values
Prev_ParentRating = dict(Prev_Exposure[['BuyerBalloon', 'PARENT_IBR_RATING']].values)
BuyerReport['Parent_IBR_Prev'] = BuyerReport['BuyerBalloon'].map(Prev_ParentRating)

New_ParentRating = dict(New_Exposure[['BuyerBalloon', 'PARENT_IBR_RATING']].values)
BuyerReport['Parent_IBR_New'] = BuyerReport['BuyerBalloon'].map(New_ParentRating)

# BalloonID 
BalloonID = dict(df[['BuyerBalloon', 'KEY']].values)
BuyerReport['Balloon_ID'] = BuyerReport['BuyerBalloon'].map(BalloonID)

# Cedant Name
Cedant = dict(df[['BuyerBalloon', 'CEDANT_NAME']].values)
BuyerReport['Cedant'] = BuyerReport['BuyerBalloon'].map(Cedant)

# Underwriter assocaited with Cedant
Underwriter = dict(TreatyRegister[['Balloon ID', 'Uwriter']].values)
BuyerReport['Underwriter'] = BuyerReport['Balloon_ID'].map(Underwriter)

# Fill blank cells in table

BuyerReport = BuyerReport.drop('BuyerBalloon', axis=1)

BuyerReport.head(50)

,Buyer_ID,Buyer_Name,Buyer_Country,Buyer_Industry,Buyer_TPE_Prev,Buyer_TPE_New,Buyer_TPE_Delta,Buyer_IBR_Prev,Buyer_IBR_New,Buyer_IBR_Delta,Parent_ID,Parent_Name,Parent_IBR_Prev,Parent_IBR_New,Parent_IBR_Delta,Balloon_ID,Cedant,Underwriter
0,23416786,BERERIX PS,Latvia,Transport,11764000,11764000,0,74,74,NaN,23416786,BERERIX PS,74,74,NaN,00794B1 01,BTA LATVIA,TMI
1,48522479,RERE GRUPA AS,Latvia,Finance,4944000,3984000,-960000,74,74,NaN,48522479,RERE GRUPA AS,74,74,NaN,00794B1 01,BTA LATVIA,TMI
2,14894648,BMGS AS,Latvia,Construction,3108000,3108000,0,56,56,NaN,14894648,BMGS AS,56,56,NaN,00794B1 01,BTA LATVIA,TMI
3,20595241,LNK (LATVIJAS NOVITATES KOMPLEKSS),Latvia,Construction,2278000,2278000,0,49,49,NaN,20595241,LNK (LATVIJAS NOVITATES KOMPLEKSS),49,49,NaN,00794B1 01,BTA LATVIA,TMI
4,15585917,LATVIJAS ENERGOCELTNIEKS SIA,Latvia,Construction,1739000,1739000,0,60,60,NaN,23263885,SKONTO GROUP SIA,48,48,NaN,00794B1 01,BTA LATVIA,TMI
5,7225297,AMBER LATVIJAS BALZAMS AS,Latvia,Chemicals,1634000,1634000,0,16,16,NaN,23006000,AMBER BEVERAGE GROUP HOLDING S.À,39,50,NaN,00794B1 01,BTA LATVIA,TMI
6,16982411,FL TECHNICS UAB,Lithuania,Transport,1436000,1436000,0,45,45,NaN,23591020,AVIA SOLUTIONS GROUP PLC,50,50,NaN,00794B1 01,BTA LATVIA,TMI
7,8952518,KESKO SENUKAI LATVIA AS,Latvia,Construction Materials,1200000,1200000,0,47,48,NaN,3844136,KESKO OYJ,24,24,NaN,00794B1 01,BTA LATVIA,TMI
8,12860943,YIT LIETUVA UAB,Lithuania,Construction,326000,326000,0,No Rating,No Rating,NaN,771827,YIT OYJ,61,61,NaN,00794B1 01,BTA LATVIA,TMI
9,15830296,LITNAGLIS UAB,Lithuania,Metals,246000,246000,0,37,43,NaN,40701755,LITMET UAB,39,33,NaN,00794B1 01,BTA LATVIA,TMI


In [97]:
# Update missing IBR ratings with existing Matching ID first
#BuyerReport["Buyer_IBR_Prev"] = BuyerReport.groupby(['Buyer_ID', 'Buyer_Name'], sort=False).bfill()['Buyer_IBR_Prev']




BuyerReport.loc[BuyerReport['Buyer_ID'] == '164336']

,Buyer_ID,Buyer_Name,Buyer_Country,Buyer_Industry,Buyer_TPE_Prev,Buyer_TPE_New,Buyer_TPE_Delta,Buyer_IBR_Prev,Buyer_IBR_New,Buyer_IBR_Delta,Parent_ID,Parent_Name,Parent_IBR_Prev,Parent_IBR_New,Parent_IBR_Delta,Balloon_ID,Cedant,Underwriter
18514,164336,BAWN DEVELOPMENTS LIMITED,Ireland,Construction,70000,64000,-6000,41,41,NaN,164336,BAWN DEVELOPMENTS LIMITED,41,41,NaN,36153B1 01,ADVENT INSURANCE DAC,CAT
21433,164336,BAWN DEVELOPMENTS LIMITED,Ireland,Construction,0,70000,70000,NaN,41,NaN,164336,BAWN DEVELOPMENTS LIMITED,NaN,41,NaN,36205B1 01,QEL,MR


Print the Buyer ID's for Buyers with missing IBR values in the Prev_IBR column. Update blank cells for buyers.

In [40]:
# Create object with the empty Buyer IBR Prev rows
# Using a for loop to print all the rows with empty IBR New Ratings
IBR_Prev_List = BuyerReport[BuyerReport['Buyer_IBR_Prev'].isna()]

for element in IBR_Prev['Buyer_ID']:
    print("'",element,"',")

' 2240559 ',
' 1020154 ',
' 14320811 ',
' 20021227 ',
' 45797074 ',
' 22901958 ',
' 24302019 ',
' 18935956 ',
' 1714472 ',
' 18830279 ',
' 17713602 ',
' 23005162 ',
' 64790922 ',
' 13868114 ',
' 11278497 ',
' 9116529 ',
' 18429089 ',
' 43859214 ',
' 23843441 ',
' 23843425 ',
' 14951388 ',
' 11155539 ',
' 16412256 ',
' 22070638 ',
' 9465448 ',
' 23979297 ',
' 24186760 ',
' 19810425 ',
' 2257350 ',
' 48789 ',
' 18522250 ',
' 21577886 ',
' 10553878 ',
' 17624072 ',
' 23563891 ',
' 16780649 ',
' 20709810 ',
' 19651656 ',
' 18062400 ',
' 20601237 ',
' 20708882 ',
' 22762316 ',
' 73150992 ',
' 10382713 ',
' 15459465 ',
' 16739281 ',
' 88171604 ',
' 19732873 ',
' 23160039 ',
' 1830844 ',
' 10527023 ',
' 19354965 ',
' 4127626 ',
' 15613512 ',
' 11483208 ',
' 4293258 ',
' 22330717 ',
' 20620821 ',
' 19410342 ',
' 4766527 ',
' 11497759 ',
' 19228424 ',
' 20112867 ',
' 24311169 ',
' 7135491 ',
' 23768530 ',
' 14116728 ',
' 22827225 ',
' 17516518 ',
' 15510000 ',
' 24056269 ',
' 20709656 ',
' 1626

In [41]:
# Create object with the empty Buyer IBR New rows
# Using a for loop to print all the rows with empty IBR New Ratings
IBR_New_List = BuyerReport[BuyerReport['Parent_IBR_New'].isna()]

for element in IBR_New['Buyer_ID']:
    print("'",element,"',")

' 19222492 ',
' 21912911 ',
' 20548780 ',
' 4283165 ',
' 1983841 ',
' 14916388 ',
' 52786111 ',
' 9763225 ',
' 36952717 ',
' 2735335 ',
' 20990580 ',
' 231579 ',
' 13727569 ',
' 24261590 ',
' 21813014 ',
' 20018073 ',
' 1625444 ',
' 32983916 ',
' 16018501 ',
' 20706742 ',
' 9935581 ',
' 8967202 ',
' 17177158 ',
' 20032500 ',
' 22365958 ',
' 23235667 ',
' 20550853 ',
' 14873015 ',
' 22037956 ',
' 22366410 ',
' 22478917 ',
' 23879716 ',
' 16373795 ',
' 22365298 ',
' 15369330 ',
' 23781608 ',
' 1487555 ',
' 19215264 ',
' 18522197 ',
' 20305926 ',
' 17291635 ',
' 18522176 ',
' 20384356 ',
' 18522206 ',
' 22828113 ',
' 16145815 ',
' 16768301 ',
' 34354162 ',
' 23540111 ',
' 2086028 ',
' 22365316 ',
' 16543927 ',
' 19132100 ',
' 18798952 ',
' 18522129 ',
' 1311624 ',
' 20805713 ',
' 15347092 ',
' 19910139 ',
' 227469 ',
' 20705940 ',
' 10334068 ',
' 41676454 ',
' 23725345 ',
' 23725345 ',
' 3909794 ',
' 46279233 ',
' 23844126 ',
' 23227050 ',
' 48199472 ',
' 16645332 ',
' 10239861 ',
' 15880

Update the IBR columns with the IBR ratings from TOAD

In [80]:
# Load files from TOAD into VS Code
Prev_Missing = pd.read_excel(r'C:\Users\iesmcs1\Desktop\Repository\Seamus\IBR JUN 30.xlsx')
New_Missing = pd.read_excel(r'C:\Users\iesmcs1\Desktop\Repository\Seamus\IBR Apr 1.xlsx')

Prev_Missing["BUYER_ID"] = Prev_Missing["BUYER_ID"].apply(str)
New_Missing["BUYER_ID"] = New_Missing["BUYER_ID"].apply(str)

Prev_Missing.head()

,BUYER_ID,BUYER_NAME,BUYER_IBR_RATING,RATING_TYPE,EXPIRY_DATE,START_DATE
0,167365,COMPANHIA SIDERURGICA NACIONAL,61.0,FINM,00:00:00,00:44:25.500000
1,289815,TET SIA,31.0,FIN,00:33:12.800000,00:20:28.400000
2,299189,SMITHS GROUP PLC,30.0,FINM,00:00:00,00:10:11.200000
3,548253,AMERICAN AIRLINES GROUP INC.,52.0,FINM,00:00:00,00:49:52.900000
4,1114471,KOBE STEEL LTD,57.0,FINM,00:00:00,00:34:31.400000


In [ ]:
# where BuyerReport.loc['Buyer_IBR_Prev'] == 0 but buyer has 2 lines of exposure


In [81]:
# Update the BuyerReport IBR values with the data in the _Missing tables. 
Prev_IBR_TOAD = dict(Prev_Missing[['BUYER_ID', 'BUYER_IBR_RATING']].values)
BuyerReport['Buyer_IBR_Prev'] = BuyerReport['BuyerBalloon'].map(Prev_Rating)
Prev_IBR_TOAD


BuyerReport['Buyer_IBR_Prev'] = np.where(BuyerReport.BUYER_ID == Prev_Missing.BUYER_ID, )

{'167365': 61.0,
 '289815': 31.0,
 '299189': 30.0,
 '548253': 52.0,
 '1114471': 57.0,
 '1417087': 55.0,
 '1565314': 46.0,
 '1625444': 55.0,
 '1625657': 38.0,
 '1629545': 48.0,
 '1671297': 16.0,
 '1782160': 41.0,
 '1841004': 44.0,
 '1983841': 43.0,
 '1999713': 33.0,
 '2037650': 33.0,
 '3848123': 41.0,
 '3848802': 43.0,
 '3867547': 49.0,
 '3890827': 32.0,
 '3909794': 57.0,
 '4099704': 39.0,
 '4211420': 37.0,
 '4311421': 22.0,
 '4772068': 44.0,
 '4772225': 14.0,
 '4868919': 24.0,
 '4870133': 60.0,
 '7010386': 19.0,
 '7116219': 30.0,
 '7243670': 43.0,
 '7266466': 56.0,
 '7310471': 51.0,
 '9763225': 100.0,
 '10239861': 33.0,
 '10379466': 58.0,
 '10554265': 19.0,
 '10597519': 63.0,
 '11079547': 32.0,
 '11196970': 44.0,
 '11201644': nan,
 '11261220': 16.0,
 '11418548': 31.0,
 '11436878': 33.0,
 '11497289': 52.0,
 '11515065': 49.0,
 '11534953': 44.0,
 '11562711': 100.0,
 '11587695': 38.0,
 '11597012': 26.0,
 '11683263': 22.0,
 '11691957': 62.0,
 '13678710': 43.0,
 '13727569': 100.0,
 '13963159

In [65]:
BuyerReport.head(50)

,Buyer_ID,Buyer_Name,Buyer_Country,Buyer_Industry,Buyer_TPE_Prev,Buyer_TPE_New,Buyer_TPE_Delta,Buyer_IBR_Prev,Buyer_IBR_New,Buyer_IBR_Delta,Parent_ID,Parent_Name,Parent_IBR_Prev,Parent_IBR_New,Parent_IBR_Delta,Balloon_ID,Cedant,Underwriter
0,23416786,BERERIX PS,Latvia,Transport,11764000,11764000,0,74,74,NaN,23416786,BERERIX PS,74,NaN,NaN,00794B1 01,BTA LATVIA,TMI
1,48522479,RERE GRUPA AS,Latvia,Finance,4944000,3984000,-960000,74,74,NaN,48522479,RERE GRUPA AS,74,NaN,NaN,00794B1 01,BTA LATVIA,TMI
2,14894648,BMGS AS,Latvia,Construction,3108000,3108000,0,56,56,NaN,14894648,BMGS AS,56,NaN,NaN,00794B1 01,BTA LATVIA,TMI
3,20595241,LNK (LATVIJAS NOVITATES KOMPLEKSS),Latvia,Construction,2278000,2278000,0,49,49,NaN,20595241,LNK (LATVIJAS NOVITATES KOMPLEKSS),49,NaN,NaN,00794B1 01,BTA LATVIA,TMI
4,15585917,LATVIJAS ENERGOCELTNIEKS SIA,Latvia,Construction,1739000,1739000,0,60,60,NaN,23263885,SKONTO GROUP SIA,48,NaN,NaN,00794B1 01,BTA LATVIA,TMI
5,7225297,AMBER LATVIJAS BALZAMS AS,Latvia,Chemicals,1634000,1634000,0,16,16,NaN,23006000,AMBER BEVERAGE GROUP HOLDING S.À,39,NaN,NaN,00794B1 01,BTA LATVIA,TMI
6,16982411,FL TECHNICS UAB,Lithuania,Transport,1436000,1436000,0,45,45,NaN,23591020,AVIA SOLUTIONS GROUP PLC,50,NaN,NaN,00794B1 01,BTA LATVIA,TMI
7,8952518,KESKO SENUKAI LATVIA AS,Latvia,Construction Materials,1200000,1200000,0,47,48,NaN,3844136,KESKO OYJ,24,NaN,NaN,00794B1 01,BTA LATVIA,TMI
8,12860943,YIT LIETUVA UAB,Lithuania,Construction,326000,326000,0,No Rating,No Rating,NaN,771827,YIT OYJ,61,NaN,NaN,00794B1 01,BTA LATVIA,TMI
9,15830296,LITNAGLIS UAB,Lithuania,Metals,246000,246000,0,37,43,NaN,40701755,LITMET UAB,39,NaN,NaN,00794B1 01,BTA LATVIA,TMI


In [7]:
# IBR values for Buyers that are absent from latest InRE Named data (Prev_Exp)
# After retrieving all the IBR Ratings from TOAD save to excel file.
# Load Excel file into VS Code. 

df1 = pd.read_excel(r'C:\Users\iesmcs1\Desktop\Repository\Seamus\TPE_New_IBR_Ratings.xlsx')
df1.head()

,BUYER_ID,BUYER_NAME,BUYER_IBR_RATING
0,27854,B S A COMPANY LTD,57.0
1,79983,TELUS CORPORATION,56.0
2,83531,CATERPILLAR INC.,30.0
3,84469,BAK AMBALAJ SANAYI VE TICARET A.S.,38.0
4,86294,DOOSAN CO LTD,60.0


In [8]:
IBR_New = dict(df1[['BUYER_ID', 'BUYER_IBR_RATING']].values)
BuyerReport['Buyer IBR New'] = BuyerReport['Buyer ID'].map(IBR_New)

BuyerReport.head(15)

,Buyer ID,Buyer Name,Buyer Country,Buyer Industry,Buyer TPE Prev,Buyer TPE New,Buyer TPE Delta,Buyer IBR Prev,Buyer IBR New,Buyer IBR Delta,Parent ID,Parent Name,Parent IBR Prev,Parent IBR New,Parent IBR Delta,Balloon ID,Cedant,Underwriter
0,48522479,RERE GRUPA AS,Latvia,Finance,4199000,4944000,745000,74,NaN,NaN,48522479,RERE GRUPA AS,74,74,NaN,00794B1 01,BTA LATVIA,MR
1,20595241,LNK (LATVIJAS NOVITATES KOMPLEKSS),Latvia,Construction,3411000,2278000,-1133000,49,NaN,NaN,20595241,LNK (LATVIJAS NOVITATES KOMPLEKSS),49,49,NaN,00794B1 01,BTA LATVIA,MR
2,15585917,LATVIJAS ENERGOCELTNIEKS SIA,Latvia,Construction,2751000,1739000,-1012000,60,NaN,NaN,23263885,SKONTO GROUP SIA,48,48,NaN,00794B1 01,BTA LATVIA,MR
3,7225297,AMBER LATVIJAS BALZAMS AS,Latvia,Chemicals,2484000,1634000,-850000,16,NaN,NaN,23006000,AMBER BEVERAGE GROUP HOLDING S.À,39,39,NaN,00794B1 01,BTA LATVIA,MR
4,14894648,BMGS AS,Latvia,Construction,2258000,3108000,850000,56,NaN,NaN,14894648,BMGS AS,56,56,NaN,00794B1 01,BTA LATVIA,MR
5,11496153,ARCERS SIA,Latvia,Construction,1809000,0,-1809000,78,NaN,NaN,11496153,ARCERS SIA,78,NaN,NaN,00794B1 01,BTA LATVIA,MR
6,22741026,TT BALTICS SIA,Latvia,Services,1200000,0,-1200000,71,NaN,NaN,22741026,TT BALTICS SIA,71,NaN,NaN,00794B1 01,BTA LATVIA,MR
7,23331203,PERSONU APVIENIBA OCT PS,Latvia,Transport,960000,960000,0,96,NaN,NaN,23331203,PERSONU APVIENIBA OCT PS,96,96,NaN,00794B1 01,BTA LATVIA,MR
8,21743712,WOLTEC SIA,Latvia,Construction,866000,703000,-163000,45,NaN,NaN,21743712,WOLTEC SIA,45,45,NaN,00794B1 01,BTA LATVIA,MR
9,49639888,BUNASTA UAB,Lithuania,Transport,864000,0,-864000,54,NaN,NaN,49639888,BUNASTA UAB,54,NaN,NaN,00794B1 01,BTA LATVIA,MR


In [9]:
for element in BuyerReport['Buyer ID']:
    BuyerReport['Buyer ID']
    print("'",element,"',")

' 48522479 ',
' 20595241 ',
' 15585917 ',
' 7225297 ',
' 14894648 ',
' 11496153 ',
' 22741026 ',
' 23331203 ',
' 21743712 ',
' 49639888 ',
' 16910343 ',
' 17439329 ',
' 15554232 ',
' 15830296 ',
' 17664876 ',
' 16738527 ',
' 17322306 ',
' 17118684 ',
' 21442788 ',
' 22488868 ',
' 14810497 ',
' 4333006 ',
' 21182202 ',
' 15941499 ',
' 21766994 ',
' 68930364 ',
' 23883361 ',
' 23572850 ',
' 17122102 ',
' 21416794 ',
' 21856442 ',
' 16787621 ',
' 23883362 ',
' 23573082 ',
' 23573084 ',
' 23573079 ',
' 33419901 ',
' 10734959 ',
' 48684289 ',
' 21442785 ',
' 17009703 ',
' 16454646 ',
' 16501368 ',
' 10464376 ',
' 22488797 ',
' 15459384 ',
' 21442818 ',
' 9021361 ',
' 20444093 ',
' 19587898 ',
' 63206009 ',
' 23573091 ',
' 871815 ',
' 93151375 ',
' 11475399 ',
' 17401150 ',
' 18304694 ',
' 20583096 ',
' 22720116 ',
' 16262885 ',
' 16524661 ',
' 86444081 ',
' 11567129 ',
' 16797555 ',
' 23883363 ',
' 15601956 ',
' 19143773 ',
' 21388488 ',
' 79109638 ',
' 17195388 ',
' 80051982 ',
' 16216615 

In [ ]:
for element, row in df.iterrows():
    if df1.loc[element,'stream'] == 2:

In [ ]:
BuyerReport.loc[BuyerReport['Buyer TPE New'] == 0, ]

In [15]:
# Buyer IBR Ratings from SQL Query
# Print all Buyer ID's for SQL Query
for element in BuyerReport['Buyer ID']:
    BuyerReport['Buyer ID']
    print("'",element,"',")

' 48522479 ',
' 20595241 ',
' 15585917 ',
' 7225297 ',
' 14894648 ',
' 11496153 ',
' 22741026 ',
' 23331203 ',
' 21743712 ',
' 49639888 ',
' 16910343 ',
' 17439329 ',
' 15554232 ',
' 15830296 ',
' 17664876 ',
' 16738527 ',
' 17322306 ',
' 17118684 ',
' 21442788 ',
' 22488868 ',
' 14810497 ',
' 4333006 ',
' 21182202 ',
' 15941499 ',
' 21766994 ',
' 68930364 ',
' 23883361 ',
' 23572850 ',
' 17122102 ',
' 21416794 ',
' 21856442 ',
' 16787621 ',
' 23883362 ',
' 23573082 ',
' 23573084 ',
' 23573079 ',
' 33419901 ',
' 10734959 ',
' 48684289 ',
' 21442785 ',
' 17009703 ',
' 16454646 ',
' 16501368 ',
' 10464376 ',
' 22488797 ',
' 15459384 ',
' 21442818 ',
' 9021361 ',
' 20444093 ',
' 19587898 ',
' 63206009 ',
' 23573091 ',
' 871815 ',
' 93151375 ',
' 11475399 ',
' 17401150 ',
' 18304694 ',
' 20583096 ',
' 22720116 ',
' 16262885 ',
' 16524661 ',
' 86444081 ',
' 11567129 ',
' 16797555 ',
' 23883363 ',
' 15601956 ',
' 19143773 ',
' 21388488 ',
' 79109638 ',
' 17195388 ',
' 80051982 ',
' 16216615 

In [93]:
# Export dataframe to excel
BuyerReport.to_excel(r'C:\Users\iesmcs1\Desktop\Repository\Seamus\BuyerReport.xlsx', sheet_name='BuyerReport', index=False)

In [16]:
# Export dataframe to excel
UW_df.to_excel(r'C:\Users\iesmcs1\Desktop\Repository\Seamus\UW_df.xlsx', sheet_name='UW_df', index=False)

In [94]:
BuyerReport.loc[BuyerReport['Buyer_ID'] == '164336']

,Buyer_ID,Buyer_Name,Buyer_Country,Buyer_Industry,Buyer_TPE_Prev,Buyer_TPE_New,Buyer_TPE_Delta,Buyer_IBR_Prev,Buyer_IBR_New,Buyer_IBR_Delta,Parent_ID,Parent_Name,Parent_IBR_Prev,Parent_IBR_New,Parent_IBR_Delta,Balloon_ID,Cedant,Underwriter
18514,164336,BAWN DEVELOPMENTS LIMITED,Ireland,Construction,70000,64000,-6000,41,41,NaN,164336,BAWN DEVELOPMENTS LIMITED,41,41,NaN,36153B1 01,ADVENT INSURANCE DAC,CAT
21433,164336,BAWN DEVELOPMENTS LIMITED,Ireland,Construction,0,70000,70000,NaN,41,NaN,164336,BAWN DEVELOPMENTS LIMITED,NaN,41,NaN,36205B1 01,QEL,MR


In [7]:
# UnderWriter filter
UW_df = BuyerReport[BuyerReport["Underwriter"] == "TMI"]

UW_df.to_excel(r'C:\Users\iesmcs1\Desktop\Repository\Seamus\UW_df.xlsx', sheet_name='UW_df', index=False)

In [15]:
#Cedant Filter
UW_df = UW_df[UW_df['Cedant'] == "BTA LATVIA"]
UW_df.head()

,Buyer ID,Buyer Name,Buyer Country,Buyer Industry,Buyer TPE Prev,Buyer TPE New,Buyer TPE Delta,Buyer IBR Prev,Buyer IBR New,Buyer IBR Delta,Parent Name,Parent IBR Prev,Parent IBR New,Parent IBR Delta,Balloon ID,Cedant,Underwriter
0,48522479,RERE GRUPA AS,Latvia,Finance,4199000.0,4199000.0,0.0,74,74,NaN,RERE GRUPA AS,74,74,NaN,00794B1 01,BTA LATVIA,TMI
1,20595241,LNK (LATVIJAS NOVITATES KOMPLEKSS),Latvia,Construction,3411000.0,3411000.0,0.0,64,49,NaN,LNK (LATVIJAS NOVITATES KOMPLEKSS),64,49,NaN,00794B1 01,BTA LATVIA,TMI
2,15585917,LATVIJAS ENERGOCELTNIEKS SIA,Latvia,Construction,2751000.0,2751000.0,0.0,59,60,NaN,SKONTO GROUP SIA,59,48,NaN,00794B1 01,BTA LATVIA,TMI
3,7225297,AMBER LATVIJAS BALZAMS AS,Latvia,Chemicals,2484000.0,2484000.0,0.0,14,16,NaN,AMBER BEVERAGE GROUP HOLDING S.À,39,39,NaN,00794B1 01,BTA LATVIA,TMI
4,14894648,BMGS AS,Latvia,Construction,2258000.0,2258000.0,0.0,56,56,NaN,BMGS AS,56,56,NaN,00794B1 01,BTA LATVIA,TMI


In [15]:
#Buyer_ws.append(list)
for i in rows:
    Buyer_ws.append([i])

IBR_wb.save("C:/Users/iesmcs1/Desktop/Repository/Seamus/IBR Ratings.xlsx")

In [16]:
# set file path
filepath="C:/Users/iesmcs1/Desktop/Repository/Seamus/IBR Ratings.xlsx"
# save workbook
# Any changes made to Excel Workbook, save wb to see the changes
IBR_wb.save(filepath)